In [293]:
import random as rn
import numpy as np
import pandas as pd
import time as ti
import itertools as it
from IPython.display import display, Latex
import scipy.special
import math

In [294]:
number_of_decks = 1#The number of decks for which you want a .csv file of all exact sticking probabilities

In [295]:
csv_label = "perfect_strategy_" + str(number_of_decks) + ".csv"
csv_label 

'perfect_strategy_1.csv'

# Optimal Stategy

This notebook's goal is calculation of the exact stick probabilities for all positions for any number of decks. 

In [296]:
full_deck = ["2","3","4","5","6","7","8","9","1","1","1","1","0"]*4*number_of_decks #1 == 10, 0 == A
options = ["2","3","4","5","6","7","8","9","1","0"]

Calculate player score for a position:

In [297]:
def score(i):
    temp = 0
    high_aces = 0
    i = list(str(i))[1:]
    i.reverse()
    for j in range(2,10):
        temp += int(i[j])*j
    high_aces = int(i[0])
    temp += 11 * int(i[0]) 
    temp += 10*int(i[1])
    while temp > 21 and high_aces > 0:
        temp -= 10
        high_aces -= 1
    return temp

In [298]:
score(10023110101)

44

This function adds pairs all possible dealer visible cards with an inputted player hand. In single deck when the player holds [2,2,2,2], the dealer can have any card but for 2 as all 2 cards are in the player's hand:

In [299]:
def dealer_card(position,my_dict):
    string = list(str(position))[2:]
    string.reverse()
    if all([int(i) < (4*number_of_decks + 1) for i in string]):
        for j in range(0,10):
            if int(string[j]) < (4*number_of_decks):
                my_dict[position + 10000000000*j] = 0
    return(my_dict)

This cell calculates every possible position for the number of decks specified at the top of this notebook:

In [300]:
start = ti.time()
baseline = 10**11
my_dict = {}
new = {}
powers = {0:1,1:10,2:100,3:1000,4:10**4,5:10**5,6:10**6,7:10**7,8:10**8,9:10**9,10:10**10}
for i in range(0,10):
    if score(baseline + powers[i]) < 22:
        new[baseline + powers[i]] = 0#not adding one card stuff
done = 0
while done == 0:
    previous = list(new.copy().keys())
    new = {}   
    for index in previous:
        cards = list(str(index))[2:]
        cards.reverse()
        for i in range(0,10):
            current_index = int(index) + powers[i]
            if score(int(current_index)) < 22:
                if (int(current_index)) not in list(my_dict.keys()):
                    if int(cards[i]) <  int(4*number_of_decks+1) and int(cards[i]) != 9:#You cant draw 10 2's or 10 aces for reaosn discussed in section 1.3 of thesis document
                        new[index + powers[i]] = 0     
                        my_dict = dealer_card(index + powers[i], my_dict)
    if len(new) == 0:
         done = 1
    print(len(new.keys()))
ti.time() - start
len(my_dict)

55
179
336
438
450
385
265
139
55
14
1
0


19620

In [301]:
len(my_dict)

19620

This cell calculates the win, loss and tie rates for the player analytically:

In [302]:
def current(index, decks):
    player_score = score(index)
    to_draw = {}
    string = list(str(index))[2:]
    string.reverse()
    for i in range(0,10):
        to_draw[i] = 4*decks - int(string[i])
    to_draw[1] += 12*decks
    #Right away derive how many of each card the dealer can draw. Just done at start so okay if slow
    powers = {0:1,1:10,2:100,3:1000,4:10**4,5:10**5,6:10**6,7:10**7,8:10**8,9:10**9,10:10**10}
    my_dict = {}#fix this
    dealer_card = int(str(index)[1])
    to_draw[dealer_card] -= 1#The dealer visible card isnt in the deck so we remove it
    cards_in_deck = sum(list(to_draw.values()))
    done = 0
    new = {10**11 + powers[dealer_card]:1}#This is the iitial dealer hand as  a player-like index. The associated likelihood if it being drawn is 1 as it is the starting position
    done = 0
    depth = 0 
    while done == 0:
        previous = new.copy()
        new = {}   
        for index in previous.keys():
            string = [int(i) for i in list(str(index))[2:]]#dealer's hand
            string.reverse()#Make it so position i describes card type i
            string[dealer_card] -= 1#Remove the dealer's intitial card as it is a guarneteed draw and is therefore not affecting the probabilities
            for i in range(0,10): # You need to go through the entire remaining deckHERES THE ISSUE
                if int(string[i]) < to_draw[i]: # Is this position possible with the deck associated with this position>?
                    current = index + powers[i] #We have a valid position with this index
                    if score(current) < 17:#If this position requires the dealer to draw again, it goes into new
                        if current not in new.keys():
                            new[current] = 0
                        new[current] += previous[index]*(to_draw[i] - int(string[i]))/(cards_in_deck - depth) #This is just the number of ways you can the draw card you want divided by total cards remaining in deck
                    
                    if current not in my_dict.keys() and score(current) > 16:#If the score is greater than 17 the delaer has to stick and the hand is finished. Find the probability of this route and sum all of the routes together
                        my_dict[current] =  0
                    if score(current) > 16:
#-----------------------------------------------------------------------------------------------
                        string2 = [int(i) for i in list(str(current))[2:]]
                        string2.reverse()
                        string2[dealer_card] -= 1
                        mult = 1
                        for j in range(0,10):
                            if string2[j] > 1:
                                mult = mult*math.factorial(string2[j])
#-----------------------------------------------------------------------------------------------
                        my_dict[current] += previous[index]*(to_draw[i] - int(string[i]))/(cards_in_deck -depth)
        depth += 1
        if len(new) == 0:
            done = 1
    count = [0,0,0]
    #Now we have all dealer positions and how many ways they can reach each.             
    for i in list(my_dict.keys()):
        dealer_score = score(i)
        if (player_score < 22):
            if (player_score > dealer_score):
                count[0] += my_dict[i]  
            if player_score < dealer_score and dealer_score < 22:
                count[1] += my_dict[i]  
            if (dealer_score > 21):
                count[0] += my_dict[i]  
            if (player_score == dealer_score):
                count[2] += my_dict[i]  
        if (player_score > 21):
            my_dict[1] += my_dict[i]  
    return(count)

In [304]:
start = ti.time()
for i in list(my_dict.keys()):
    my_dict[i] = current(i, 2)
ti.time() - start

457.1433663368225

The data is now transformed so it is suitable for the R package's functions that use it.

In [305]:
minimum = 0
scores = {}
my_dict2 = {}
labels = {}
for i in list(my_dict.keys()):
    my_dict2[i] = (my_dict[i][0] - my_dict[i][1])/(my_dict[i][0] + my_dict[i][1] + my_dict[i][2])#Expected return
    scores[i] = score(i)
    labels[i] = str(i)

Convert data to data frame and output to csv.

In [307]:
#pd.DataFrame({"label" :labels.keys(), "Stick Expected Return": my_dict2.values(), "Scores":scores.values()}).to_csv(csv_label)